In [36]:
print('Hello!')
import os
os.chdir('..')
from dotenv import load_dotenv
load_dotenv()

Hello!


True

##### Instructor Prototype 

In [188]:
from pydantic import BaseModel,Field, ValidationError, model_validator, field_validator
from enum import Enum
from typing import Union, Optional


class RequestCategory(str, Enum):
    VOICE_COMMAND = "voice_command"
    MUSIC_SUGGESTION = "music_suggestion"

class MusicGenre(str, Enum):
    TECHNO = 'techno'
    REGGAE = 'reggae'

class Command(str, Enum):
    BGCOLOR1 = "set_bg_color1"
    SPAWNASYNC = "spawn"
    SHOWBLOCKLIST = 'show_block_list'

class ColorArgument(str, Enum):
    BLUE = 'blue'
    RED = 'red'
    GREEN = 'green'
    PURPLE = 'purple'

class RGBColor(BaseModel):
    r: float = Field(..., description="Red component of the color (0.0 to 1.0)")
    g: float = Field(..., description="Green component of the color (0.0 to 1.0)")
    b: float = Field(..., description="Blue component of the color (0.0 to 1.0)")

    # description: str = "Color should be provided as an RGB tuple with each value between 0.0 and 1.0, e.g., [0.5, 0.2, 0.8]"

    # Custom Validator to enforce the RGB range
    @field_validator('r', 'g', 'b')
    def validate_color_range(cls, value):
        if not (0.0 <= value <= 1.0):
            raise ValueError('Color values must be between 0.0 and 1.0')
        return value

# class RGBColor(BaseModel):
#     r: float = Field(..., ge=0.0, le=1.0, description="Red component of the color (0.0 to 1.0)")
#     g: float = Field(..., ge=0.0, le=1.0, description="Green component of the color (0.0 to 1.0)")
#     b: float = Field(..., ge=0.0, le=1.0, description="Blue component of the color (0.0 to 1.0)")


#     # @field_validator('r', 'g', 'b')
#     # def validate_color_range(cls, value):
#     #     if not (0.0 <= value <= 1.0):
#     #         raise ValueError('Color values must be between 0.0 and 1.0')
#     #     return value


class SpawnArgument(str, Enum):
    OSC = 'oscillator'
    SPEAKER = 'speaker'
    NOIZE = 'noize'
    VIOLIN = 'violin'


In [189]:
class VoiceCommandArguments(BaseModel):
    command: Command
    arguments: Optional[Union[RGBColor, SpawnArgument]] = None

    @model_validator(mode='after')
    def validate_command_arguments(self):
        if self.command == Command.BGCOLOR1 and not isinstance(self.arguments, RGBColor):
            raise ValueError(f"{self.command} requires a RGBColor.")
        if self.command == Command.SPAWNASYNC and not isinstance(self.arguments, SpawnArgument):
            raise ValueError(f"{self.command} requires a SpawnArgument.")
        if self.command == Command.SHOWBLOCKLIST and self.arguments is not None:
            raise ValueError(f"{self.command} does not accept arguments.")
        return self

class MusicSuggestionArguments(BaseModel):
    genre: MusicGenre
    mood: Optional[str] = Field(None, description="Optional mood descriptor for the suggestion.")


In [190]:
class ExtractIntention(BaseModel):
    intention: RequestCategory
    details: Union[VoiceCommandArguments, MusicSuggestionArguments]

    @model_validator(mode='after')
    def validate_intention(self):
        if self.intention == RequestCategory.VOICE_COMMAND and not isinstance(self.details, VoiceCommandArguments):
            raise ValueError(f"{self.intention} requires VoiceCommandArguments.")
        if self.intention == RequestCategory.MUSIC_SUGGESTION and not isinstance(self.details, MusicSuggestionArguments):
            raise ValueError(f"{self.intention} requires MusicSuggestionArguments.")
        return self


In [191]:
valid_voice_command = {
    "intention": "voice_command",
    "details": {
        "command": "set_bg_color1",
        # "arguments": "blue"
        # "arguments": "violin"
        "arguments": {"r": 0.5, "g": 0.2, "b": 0.8}
    }
}

valid_music_suggestion = {
    "intention": "music_suggestion",
    "details": {
        "genre": "techno",
        "mood": "energetic"
    }
}

voice_command = ExtractIntention(**valid_voice_command)
# music_suggestion = ExtractIntention(**valid_music_suggestion)


In [192]:
# from openai import OpenAI
from openai import AsyncOpenAI

client = AsyncOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

async def classify(text: str) -> str:
    # response = client.chat.completions.create(
    response = await client.with_options(max_retries=3).beta.chat.completions.parse(
        model="gpt-4o-mini", 
        response_format = ExtractIntention,
        temperature=0,
        # max_retries=4,
        messages = [{
            "role": "system",
            "content": "Analyze the user's text and extract the information.",
        },
        {
            "role": "user",
            "content": text
        }
        ]
    )
    return response

In [193]:
jb = await classify('Set BGColor to purple')
print('Something')

Something


In [195]:
jb.choices[0].message.to_dict()['content']

'{"intention":"voice_command","details":{"command":"set_bg_color1","arguments":{"r":0.5,"g":0,"b":0.5}}}'

In [197]:
import json
output_dict = json.loads(jb.choices[0].message.to_dict()['content'])
output_dict

{'intention': 'voice_command',
 'details': {'command': 'set_bg_color1',
  'arguments': {'r': 0.5, 'g': 0, 'b': 0.5}}}

In [163]:
valid_prompts = [
    "set background to blue",                # Matches Command.BGCOLOR1 with ColorArgument.BLUE
    "change background color to green",     # Matches Command.BGCOLOR1 with ColorArgument.GREEN
    "spawn an oscillator",                  # Matches Command.SPAWNASYNC with SpawnArgument.OSC
    "create a new violin sound",            # Matches Command.SPAWNASYNC with SpawnArgument.VIOLIN
    "show me the block list",               # Matches Command.SHOWBLOCKLIST
    "suggest some techno music",            # Matches MusicSuggestionArguments with MusicGenre.TECHNO
    "recommend reggae tracks",              # Matches MusicSuggestionArguments with MusicGenre.REGGAE
    "spawn a speaker",                      # Matches Command.SPAWNASYNC with SpawnArgument.SPEAKER
    "set background to red",                # Matches Command.BGCOLOR1 with ColorArgument.RED
    "play some techno with energy",         # Matches MusicSuggestionArguments with MusicGenre.TECHNO + mood
    "play reggae music for relaxing",       # Matches MusicSuggestionArguments with MusicGenre.REGGAE + mood
    "spawn some noize",                     # Matches Command.SPAWNASYNC with SpawnArgument.NOIZE
    "set the background to purple",         # Matches Command.BGCOLOR1 with ColorArgument.PURPLE
    "suggest some energetic techno songs",  # Matches MusicSuggestionArguments with MusicGenre.TECHNO
    "recommend a playlist for reggae",      # Matches MusicSuggestionArguments with MusicGenre.REGGAE
]
invalid_prompts = [
    "set background to yellow",             # Invalid: "yellow" is not a valid ColorArgument
    "spawn a guitar",                       # Invalid: "guitar" is not a valid SpawnArgument
    "play classical music",                 # Invalid: "classical" is not a valid MusicGenre
    "set background to blue and green",     # Invalid: Only one ColorArgument is allowed
    "change the background to something cool", # Invalid: Argument not specific
    "spawn a new background",               # Invalid: Mismatched Command with invalid argument
    "recommend some songs",                 # Invalid: Missing MusicGenre
    "create noise",                         # Invalid: Ambiguous, "create" not defined as a valid Command
    "show block list now",                  # Invalid: Unrecognized format for Command.SHOWBLOCKLIST
    "set color to turquoise",               # Invalid: "turquoise" is not a valid ColorArgument
    "spawn an animal",                      # Invalid: "animal" is not a valid SpawnArgument
    "play energetic music",                 # Invalid: Missing MusicGenre
    "recommend something fun",              # Invalid: Ambiguous and non-specific
    "background to green and spawn violin", # Invalid: Multiple commands in one request
    "spawn an osc and speaker",             # Invalid: Multiple SpawnArguments not supported
]


In [165]:
for prompt in valid_prompts:
    res = await classify(prompt)
    print(f"{prompt}: {res.choices[0].message.content}")

set background to blue: {"intention":"voice_command","details":{"command":"set_bg_color1","arguments":{"r":0,"g":0,"b":1}}}
change background color to green: {"intention":"voice_command","details":{"command":"set_bg_color1","arguments":{"r":0,"g":1,"b":0}}}
spawn an oscillator: {"intention":"voice_command","details":{"command":"spawn","arguments":"oscillator"}}
create a new violin sound: {"intention":"voice_command","details":{"command":"spawn","arguments":"violin"}}
show me the block list: {"intention":"voice_command","details":{"command":"show_block_list","arguments":null}}
suggest some techno music: {"intention":"music_suggestion","details":{"genre":"techno","mood":null}}
recommend reggae tracks: {"intention":"music_suggestion","details":{"genre":"reggae","mood":null}}
spawn a speaker: {"intention":"voice_command","details":{"command":"spawn","arguments":"speaker"}}
set background to red: {"intention":"voice_command","details":{"command":"set_bg_color1","arguments":{"r":1,"g":0,"b":

In [166]:
for prompt in invalid_prompts:
    res = await classify(prompt)
    print(f"{prompt}: {res.choices[0].message.content}")

set background to yellow: {"intention":"voice_command","details":{"command":"set_bg_color1","arguments":{"r":1,"g":1,"b":0}}}
spawn a guitar: {"intention":"voice_command","details":{"command":"spawn","arguments":"violin"}}
play classical music: {"intention":"music_suggestion","details":{"genre":"techno","mood":null}}
set background to blue and green: {"intention":"voice_command","details":{"command":"set_bg_color1","arguments":{"r":0.0,"g":1.0,"b":0.0}}}
change the background to something cool: {"intention":"voice_command","details":{"command":"set_bg_color1","arguments":{"r":0.0,"g":0.0,"b":1.0}}}
spawn a new background: {"intention":"voice_command","details":{"command":"spawn","arguments":"noize"}}
recommend some songs: {"intention":"music_suggestion","details":{"genre": "techno","mood": null}}
create noise: {"intention":"voice_command","details":{"command":"spawn","arguments":"noize"}}
show block list now: {"intention":"voice_command","details":{"command":"show_block_list","argument

In [ ]:
b = await classify('Set BGColor to orange')

In [16]:
COMMAND_ALIASES = {
    "background": "set_bg_color1",
    "bg": "set_bg_color1",
    "create": "spawn",
    "give me": "spawn",
    "show list of blocks": "show_block_list",
    "blocklist": "show_block_list",
    "block list": "show_block_list",
    "blocks list": "show_block_list",
}

In [ ]:
def normalize_command(user_input: str) -> Optional[str]:
    # Normalize input to lowercase and remove extra spaces
    user_input = user_input.lower().strip()
    # Find the matching command
    for alias, command in COMMAND_ALIASES.items():
        if alias in user_input:
            return command
    return None  # Return None if no match is found


##### Exp Instructor

In [ ]:
import instructor
from openai import OpenAI

client = instructor.from_openai(OpenAI(), mode=instructor.Mode.TOOLS)

In [ ]:
from pydantic import BaseModel,Field, ValidationError
from enum import Enum
from typing import Union, Optional



In [7]:
class RequestCategory(str, Enum):
    VOICE_COMMAND = "voice_command"
    MUSIC_SUGGESTION = "music_suggestion"


In [ ]:
class LizaCommand(str, Enum):
    BGCOLOR1 = 'set_bg_color1'
    BGCOLOR2 = 'set_bg_color2'
    BGCOLORHSV = 'set_bg_color_hsv'
    # SETCOLORPALETTE = 'set_color_palette'
    SPAWNASYNC = 'spawn'
    SHOWBLOCKLIST = 'show_block_list'


In [10]:
class LizaCommand(str, Enum):
    BGCOLOR1 = 'set_bg_color1'
    SPAWNASYNC = 'spawn'
    SHOWBLOCKLIST = 'show_block_list'


In [ ]:
class SpawnArgument(str, Enum):
    OSC = 'oscillator'
    SPEAKER = 'speaker'
    NOIZE = 'noize'

In [ ]:
class RequestCategory(str, Enum):
    VOICE_COMMAND = "voice_command"
    MUSIC_SUGGESTION = "music_suggestion"

class MusicGenre(str, Enum):
    TECHNO = 'techno'
    REGGAE = 'reggae'

class Command(str, Enum):
    BGCOLOR1 = "set_bg_color1"
    SPAWNASYNC = "spawn"
    SHOWBLOCKLIST = 'show_block_list'

class ColorArgument(str, Enum):
    BLUE = 'blue'
    RED = 'red'
    GREEN = 'green'
    PURPLE = 'purple'

class SpawnArgument(str, Enum):
    OSC = 'oscillator'
    SPEAKER = 'speaker'
    NOIZE = 'noize'
    VIOLIN = 'violin'

class ExtractIntention(BaseModel):
    intention: RequestCategory
    # age: int

In [ ]:
# classify the intention:
# - voice command execution -> chat
# - music making suggestion -> chat
# - question about the docs -> RAG
# - recalling memories -> history 
# - generation of the midi -> midi gen
# - changing of the midi -> midi change module
# - generation of the sound -> sound gen module